# Joint Feldman DKG

as seen in [Secure Distributed Key Generation
for Discrete-Log Based Cryptosystems](https://link.springer.com/content/pdf/10.1007/3-540-48910-X_21.pdf)

## First part



In [1]:
import random as rnd #not secure, use os.random() instead

In [2]:
def create_polynomial(t,mod):
    coeff = [rnd.randint(1,mod-1) for _ in range(t+1)]
    return coeff

def create_commits(coeff,t,g,mod):
    comms = [pow(g,coeff[k],mod) for k in range(t+1)]
    return comms


In [3]:
def evaluate_polynomial(coeff,val,mod):
    s = 0
    for i in range(len(coeff)):
        s += (coeff[i]*pow(val,i)) %mod
        s=s%mod
    
    return s

In [4]:
def create_shares(coeff,n,mod,i):
    """
    Every party i creates shares for all j
    """
    evals = []
    for j in range(0,n):
        if j != i:
            val = evaluate_polynomial(coeff, j,mod)
            evals.append(val % mod)
        else:
            evals.append(0)
    return evals

In [5]:
def _verify_share(share, comms, j, g, t, mod):
    """
    Given a share from party i and the commitment from that party,
    this function verifies that the share is valid.
    """
    flag = False

    # Compute G = g^share mod mod
    G = pow(g, share,mod)

    # Initialize the product s to 1
    s = 1

    # Compute the product of C^j^k for k in [0, t]
    for k in range(t+1):
        jk = pow(j, k)
        Cijk = pow(comms[k], jk,mod)
        s *= Cijk
        s = s % mod

    # Check if G == s, and set the flag accordingly
    if G == s:
        # print("yeah!")
        flag = True
    else:
        print(f"G={G}, s={s}")

    return flag



def verify_shares(shares,comms,j,g,mod,n,t):
    j_share=0
    j_QUAL = [j] # each party will assume it is right
    
    for i in range(n):
        if i != j:
            share = shares['P_'+str(i)][j]
            comms_i = comms["P_"+str(i)]
            if _verify_share(share,comms_i,j, g,t, mod): 
                j_QUAL.append(i)
    
    j_QUAL.sort()
    return (j,j_QUAL)

In [6]:
t=5; n=10
a=0; b=100
g = 3; 
p=23
q=11 # q|p-1, see https://crypto.stackexchange.com/a/84384/63690
#r = 6361

In [7]:
coeff = {}
for i in range(n):
    coeff["P_"+str(i)] = create_polynomial(t,q)

print("coeff")
coeff

coeff


{'P_0': [6, 3, 9, 8, 5, 8],
 'P_1': [8, 5, 9, 10, 4, 3],
 'P_2': [8, 5, 5, 6, 7, 5],
 'P_3': [4, 1, 6, 3, 6, 4],
 'P_4': [10, 3, 6, 1, 7, 8],
 'P_5': [3, 1, 1, 5, 10, 8],
 'P_6': [5, 5, 9, 8, 1, 10],
 'P_7': [6, 8, 5, 1, 3, 5],
 'P_8': [3, 3, 4, 1, 10, 1],
 'P_9': [7, 10, 7, 4, 10, 2]}

In [8]:
comms = {}
for i in range(n):
    comms["P_"+str(i)] = create_commits(coeff["P_"+str(i)],t,g,p)
comms

{'P_0': [16, 4, 18, 6, 13, 6],
 'P_1': [6, 13, 18, 8, 12, 4],
 'P_2': [6, 13, 13, 16, 2, 13],
 'P_3': [12, 3, 16, 4, 16, 12],
 'P_4': [8, 4, 16, 3, 2, 6],
 'P_5': [4, 3, 3, 13, 8, 6],
 'P_6': [13, 13, 18, 6, 3, 8],
 'P_7': [16, 6, 13, 3, 4, 13],
 'P_8': [4, 4, 12, 3, 8, 3],
 'P_9': [2, 8, 2, 12, 8, 9]}

In [9]:
shares = {}
for i in range(n):
    shares["P_"+str(i)] = create_shares(coeff["P_"+str(i)],n,q,i)

print("shares:")
shares

shares:


{'P_0': [0, 6, 8, 10, 4, 1, 1, 7, 6, 5],
 'P_1': [8, 0, 8, 8, 2, 7, 3, 7, 4, 10],
 'P_2': [8, 3, 0, 10, 1, 8, 10, 8, 9, 10],
 'P_3': [4, 2, 3, 0, 10, 9, 0, 6, 5, 3],
 'P_4': [10, 2, 9, 4, 0, 8, 9, 10, 4, 8],
 'P_5': [3, 6, 3, 0, 7, 0, 8, 3, 5, 1],
 'P_6': [5, 5, 0, 1, 1, 9, 0, 6, 3, 4],
 'P_7': [6, 6, 5, 9, 9, 9, 10, 0, 7, 0],
 'P_8': [3, 0, 5, 6, 9, 4, 8, 9, 0, 1],
 'P_9': [7, 7, 3, 8, 7, 10, 6, 5, 3, 0]}

In [10]:
for j in range(n):
    print(verify_shares(shares,comms,j,g,p,n,t))

(0, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(1, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(2, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(3, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(4, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(5, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(6, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(7, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(8, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
(9, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


## Exercises

1. What happens if we use only one large prime? In other words, what if `p == q` ?

Answer:

2. Write a function such that given two primes `p` and `q` returns `True` if they are "good primes" (as defined in the linked paper at the beginning) and `False` otherwise. Assume `p` and `q` are prime

In [11]:
def check(p,q):
    pass

3. Complete Steps 3 and 4 of Feldman DKG as presented in linked paper at the beginning (page 6). Specifically:
    - STEP 3: identify the `QUAL` nodes (hint: use the ideas of the next cell)
    - STEP 4: compute the public key for each party

In [12]:
a = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
b = [0, 1, 3, 4, 5, 6, 7, 8, 9]
c = [0, 1, 2, 3, 4, 5, 6, 7, 9]
d = [0, 1, 2, 4, 5, 6, 7, 9]

A = [set(a),set(b),set(c),set(d)]

for i in range(1,len(A)):
    A[0] = A[0].intersection(A[i])
    
print(A[0])

{0, 1, 4, 5, 6, 7, 9}


4. Describe why the Feldman DKG is insecure (hint: argument in your own words the explanation given in the linked paper at the beginning)

Answer: